## About

- Hyper opt using Katib

In [ ]:
#yaml file to tune ml model with two hyperparams i.e learning rate and hidden units

apiVersion: "kubeflow.org/v1beta1"
kind: Experiment
metadata:
  name: example-hpo
spec:
  objective:
    type: maximize
    goal: 0.99  # The goal is to maximize the model's accuracy, for example.
  algorithm:
    algorithmName: random
  parallelTrialCount: 5  # Number of trials to run concurrently
  maxTrialCount: 10  # Total number of trials to run
  maxFailedTrialCount: 3  # Number of allowed unsuccessful trials
  parameters:
    - name: learning_rate
      parameterType: double
      feasibleSpace:
        min: "0.01"
        max: "0.1"
    - name: hidden_units
      parameterType: int
      feasibleSpace:
        min: "32"
        max: "256"
  trialTemplate:
    goTemplate:
      rawTemplate: |
        apiVersion: "kubeflow.org/v1"
        kind: TFJob
        metadata:
          name: {{.Trial}}
        spec:
          tfReplicaSpecs:
            Chief:
              replicas: 1
              template:
                spec:
                  containers:
                    - name: tensorflow
                      image: "your-custom-tensorflow-image"
                      command:
                        - "python"
                        - "/path/to/train_script.py"
                        - "--learning_rate={{.HyperParameters.learning_rate}}"
                        - "--hidden_units={{.HyperParameters.hidden_units}}"


The yaml config can be applied to the kubernetes cluster running katib to initiate and manage hyperparam tuning jobs for ml model